Il faudrait extraire du classeur de Fanny, les quantités par prix et par tunnel.
Cela devrait être possible en lisant les formules dans les onglets.
Pour le faire, il semble que openpyxl est une solution.

In [99]:
import pandas as pd
import numpy as np
from openpyxl import Workbook, load_workbook

prodQuantAnLieuUniTyp2324 = pd.read_csv('prodQuantAnLieuUniTyp2324.csv')

#### Quant_23_24

In [9]:
prodQuantAnLieuUniTyp2324.iloc[:, 4].unique()

array(['Chennevières', 'Antony', 'Bicêtre', 'Landy',
       'Nanterre / La Défense', 'PCTT', 'Nogent', 'Saint Cloud',
       'Taverny', 'Boissy-Saint-Léger', 'Bobigny-Lumen-Norton',
       'La Courneuve', 'Thiais', 'Orly', 'Neuilly', 'Ambroise PARE',
       'Bellerive', 'Fresnes', 'Italie', 'Champigny', 'Sévines',
       'Fontenay le Fleury', 'Tous tunnels', 'SIRIUS',
       'SIRIUS et TUNNELS', 'SIREDO'], dtype=object)

In [125]:
grouped = prodQuantAnLieuUniTyp2324.groupby(['lieu', 'com_bud_annee'])['com_prod_quantite'].sum().reset_index()
quant_23_24 = grouped.pivot(index='lieu', columns='com_bud_annee', values='com_prod_quantite').reset_index()

quant_23_24.columns.name = None
quant_23_24 = quant_23_24.rename(columns={2023: '2023', 2024: '2024'})

lieux_non_tunnel = ['SIREDO', 'SIRIUS', 'SIRIUS et TUNNELS', 'Thiais', 'Tous tunnels', 'PCTT']
autres_lieux = quant_23_24[quant_23_24['lieu'].isin(lieux_non_tunnel)].copy()

quant_23_24 = quant_23_24[~quant_23_24['lieu'].isin(lieux_non_tunnel)].copy()

lieux_renomé = {
    'Saint Cloud': 'Saint-Cloud',
    'Ambroise PARE': 'Ambroise Paré',
    'Bellerive': 'Belle-Rive',
    'Fontenay le Fleury': 'Fontenay',
    'Boissy-Saint-Léger': 'Boissy'
}

quant_23_24['lieu'] = quant_23_24['lieu'].replace(lieux_renomé)
quant_23_24['2023'] = quant_23_24['2023'].astype(int)
quant_23_24['2024'] = quant_23_24['2024'].astype(int)

quant_23_24.reset_index(drop=True)

print(quant_23_24)

                     lieu     2023    2024
0           Ambroise Paré    71422  364512
1                  Antony    53594  116096
2              Belle-Rive   168093  269008
3                 Bicêtre   110229  206109
4    Bobigny-Lumen-Norton    50738  272682
5                  Boissy    70001   46593
6               Champigny   156527  151724
7            Chennevières    35543   66951
8                Fontenay     4224   32115
9                 Fresnes     9137   71610
10                 Italie    11848   17632
11           La Courneuve    13518   25913
12                  Landy    94781  181544
13  Nanterre / La Défense  1195273  622045
14                Neuilly    54474   42182
15                 Nogent   433538  348774
16                   Orly    39331   58817
21            Saint-Cloud    87437  149336
22                Sévines    50614   67328
23                Taverny     6241   22330


#### Quant_25

In [100]:
classeur_path = "Tunnels_Cout préventif_04.04.2024.xlsx"
classeur = pd.read_excel(classeur_path, sheet_name=None)


tunnels_25 = [
    "Boissy", "Champigny", "Guy Môquet", "Moulin", "Nogent", "Ambroise Paré", "Belle-Rive",
    "Chennevières", "Fontenay", "La Défense", "Nanterre Centre", "Nanterre échangeur",
    "Neuilly", "Saint-Cloud", "Sévines", "Bobigny", "La Courneuve", "Landy", "Lumen-Norton",
    "Taverny", "Antony", "Fresnes", "Bicêtre", "Italie", "Orly"
]

In [ ]:
quant_data = []

for tunnel in tunnels_25:
    try:
        df_sheet = pd.read_excel(classeur_path, sheet_name=tunnel, header=1)
        df_sheet.columns = df_sheet.columns.str.strip()

        if "Fréquence\ntotale" in df_sheet.columns and "Nombre\néquipé" in df_sheet.columns:
            product = (df_sheet["Fréquence\ntotale"] * df_sheet["Nombre\néquipé"]).sum()
            quant_data.append({"lieu": tunnel, "2025": product})

        else:
            quant_data.append({"lieu": tunnel, "2025": None})
            print(f"[!] Columns not found in sheet: {tunnel} -> {df_sheet.columns.tolist()}")

    except Exception as e:
        quant_data.append({"lieu": tunnel, "2025": None})
        print(f"[!] Failed to process sheet: {tunnel} — {str(e)}")

quant_25 = pd.DataFrame(quant_data)

nanterre_ladefense_rows = quant_25[quant_25['lieu'].isin(['Nanterre Centre', 'Nanterre échangeur', 'La Défense'])]
combined_value = nanterre_ladefense_rows['2025'].sum()
combined_row = pd.DataFrame([{"lieu": "Nanterre / La Défense", "2025": combined_value}])
quant_25 = quant_25[~quant_25['lieu'].isin(['Nanterre Centre', 'Nanterre échangeur', 'La Défense'])]

quant_25 = pd.concat([quant_25, combined_row], ignore_index=True)
quant_25 = quant_25.sort_values(by='lieu', ascending=True).reset_index(drop=True)

bobigny_lumen_rows = quant_25[quant_25['lieu'].isin(['Bobigny', 'Lumen-Norton'])]
combined_value_bobigny_lumen = bobigny_lumen_rows['2025'].sum()
combined_row_bobigny_lumen = pd.DataFrame([{"lieu": "Bobigny-Lumen-Norton", "2025": combined_value_bobigny_lumen}])
quant_25 = quant_25[~quant_25['lieu'].isin(['Bobigny', 'Lumen-Norton'])]
quant_25 = pd.concat([quant_25, combined_row_bobigny_lumen], ignore_index=True)

quant_25 = quant_25[~quant_25['lieu'].isin(['Moulin', 'Guy Môquet'])]

quant_25 = quant_25.sort_values(by='lieu', ascending=True).reset_index(drop=True)

# GUY MOQUET, MOULIN, LUMEN NORTON DONT HAVE A VALUE IN PRODQUANTANLIEUUNITYP2324.csv

,lieu,2025
0,Ambroise Paré,182.300000
1,Antony,184.300000
2,Belle-Rive,178.800000
3,Bicêtre,208.300000
4,Bobigny-Lumen-Norton,351.766667
5,Boissy,157.800000
6,Champigny,165.300000
7,Chennevières,98.800000
8,Fontenay,126.800000
9,Fresnes,142.800000


In [126]:
quant_23_24_25 = pd.merge(quant_23_24, quant_25[['lieu', '2025']], on='lieu', how='left')

In [127]:
quant_23_24_25

,lieu,2023,2024,2025
0,Ambroise Paré,71422,364512,182.300000
1,Antony,53594,116096,184.300000
2,Belle-Rive,168093,269008,178.800000
3,Bicêtre,110229,206109,208.300000
4,Bobigny-Lumen-Norton,50738,272682,351.766667
5,Boissy,70001,46593,157.800000
6,Champigny,156527,151724,165.300000
7,Chennevières,35543,66951,98.800000
8,Fontenay,4224,32115,126.800000
9,Fresnes,9137,71610,142.800000
